In [2]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings(category=FutureWarning, action="ignore")

In [3]:
NUM_PV = 1

pv_dfs = [
    pd.read_csv(f"../../data/renewable/renewable{i+1}.csv.gz", compression='gzip', index_col=0)
    for i in range(1)
]

price_df = pd.read_csv("data/price.csv")
price_df = price_df.rename(columns={"TIME": "ds", "PRICE": "y"})
price_df['ds'] = pd.to_datetime(price_df['ds'])
price_df = price_df.iloc[:pv_dfs[0].shape[0]]

for i in range(len(pv_dfs)):
    pv_df = pv_dfs[i]
    pv_df["ds"] = price_df["ds"]
    pv_df = pv_df[["ds", "0"]].rename(columns={"0": "y"})
    pv_dfs[i] = pv_df

print(price_df.head())
print(price_df.shape)

print(pv_dfs[0].head())
print(pv_dfs[0].shape)


                   ds      y
0 2024-01-01 00:00:00  63.33
1 2024-01-01 00:15:00  60.02
2 2024-01-01 00:30:00  56.71
3 2024-01-01 00:45:00  53.40
4 2024-01-01 01:00:00  50.09
(8760, 2)
                   ds    y
0 2024-01-01 00:00:00  0.0
1 2024-01-01 00:15:00  0.0
2 2024-01-01 00:30:00  0.0
3 2024-01-01 00:45:00  0.0
4 2024-01-01 01:00:00  0.0
(8760, 2)


In [4]:
METRICS = ["MAE", "RMSE"]
METRICS_VAL = ["MAE_val", "RMSE_val"]

def cv_train(df):
    folds = NeuralProphet(n_lags=8, n_forecasts=8)\
        .crossvalidation_split_df(df, freq='15min', k=5, fold_pct=0.2, fold_overlap_pct=0.5)

    metrics_train = pd.DataFrame(columns=METRICS)
    metrics_test = pd.DataFrame(columns=METRICS_VAL)

    for df_train, df_test in folds:
        m = NeuralProphet(n_lags=8, n_forecasts=8)
        train = m.fit(df=df_train, freq="15min", validation_df=df_test)
        test = m.test(df=df_test)
        metrics_train = metrics_train.append(train[METRICS].iloc[-1])
        metrics_test = metrics_test.append(test[METRICS_VAL].iloc[-1])

    print(metrics_test.describe().loc[["mean", "std", "min", "max"]])

    return metrics_train, metrics_test

def train(df, split=0.67):
    df_train, df_test = train_test_split(df, test_size=split, shuffle=False)
    m = NeuralProphet(n_lags=8, n_forecasts=8)
    train_res = m.fit(df=df_train, freq="15min", validation_df=df_test)
    return m, train_res

def predict(model, df, forecast_hours=24):
    periods = int((forecast_hours * 60) / 15)
    future = model.make_future_dataframe(df, periods=periods)

    forecast = model.predict(future)
    return forecast